# Deep Research Bot
This is not the main code. It is just a playground to test stuff.

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
## Creating an agent that will search the web given a topic and return back summary.

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

# Few things to know about this agent call:
# 1. For the tool, I am using WebSearchTool and setting the search_context_size parameter to low to make the API call economical.
# 2. Another parameter called model_settings was used and in this tool_choice is passed as required which forces the agent to use this tool whenever called.
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
# Testing Agent to see if its working.

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of September 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and supporting complex interaction logic. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to design adaptive AI agents for dynamic environments. PCF enables real-time parameter reconfiguration through combinatorial spaces, supporting scalable and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system that introduces a standardized Agent-to-Agent communication layer based on the Model Context Protocol (MCP). It incorporates an Experience Pack architecture for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness through novel strategies. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **Amazon Bedrock AgentCore**: Introduced by AWS, AgentCore simplifies the development and deployment of advanced AI agents. It offers modular services supporting the full production lifecycle, including scalable serverless deployment and context management. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Microsoft Semantic Kernel**: An open-source framework that integrates AI models into applications, supporting multiple programming languages and emphasizing task orchestration and real-time analytics. ([aaditech.com](https://aaditech.com/2025/01/08/ai-agent-frameworks-for-2025/?utm_source=openai))

- **OpenAI Swarm**: Focuses on multi-agent collaboration, enabling AI agents to connect with external sources, coordinate, and tackle complex processes. It utilizes OpenAI's language models for advanced decision-making abilities. ([signitysolutions.com](https://www.signitysolutions.com/blog/top-ai-agent-frameworks?utm_source=openai))

- **Kruti**: A multilingual AI agent developed by Ola Krutrim, designed to perform real-world tasks by integrating directly with various online services. It supports text and voice in 13 Indian languages and is optimized for smartphone usage. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the evolution of intelligent, autonomous systems across various applications. 

In [5]:
# WebSearchTool is generally expensive compared to the LLM API calls limitting the search to 3 for now.

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# This basically tells the agent how the output is expected from the agent.
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
# Testing Agent to see if its working.

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current frameworks for AI agents that are being adopted in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gather information about popular AI frameworks used by developers and businesses for creating AI agents in 2025.', query='top AI frameworks for agents 2025'), WebSearchItem(reason='To explore emerging technologies and trends in AI agents as of 2025.', query='emerging AI trends frameworks 2025')]


In [7]:
## Creating the function as a tool using a decorator so that it can be passed to the agent for use.
## This is just taking the output of research from other agent as subject and Body of an email so that it can be sent to us using sendgrid API.

@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("shivam.parashar2015@vitalum.ac.in")
    to_email = To("sspador@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110dac540>, strict_json_schema=True, is_enabled=True)

In [9]:
## Creating an email agent which can send a nicely formatted email using the above created tools.

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [10]:
## Creating another agent who works as a senior researcher and performs the role mentioned in INSTRUCTIONS variable.

INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent.

In [11]:
## Creating 3 coroutines functions.

# This method simply plan searches by using the planner agent created earlier and returning the results.
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

# This method perform search based on the search_plan passed which is created as an object of WebSearchPlan class created when planner agent is called.
async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

# This method is just going to use the search agent created earlier on each WebSearchItem object which is part of a WebSearchPlan.
async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it
Note: Sending reports in email for now. I will later create some UI to use it as a tool.

In [12]:
# This method uses a writer agent created earlier to write the content of the report. It takes query returned by plan_seaerches and search_results returned by searches.
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

# This method just sends the email using email agent created earlier.
async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [13]:
## Results

query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### Took a long time but recieved an email with the final report.